In [67]:
import numpy as np
from scipy import linalg

In [30]:
matrix_A = np.array([[31 ,-13 ,0 ,0 ,0 ,-10 ,0 ,0 ,0 ],
[-13 ,35 ,-9 ,0 ,-11 ,0, 0, 0, 0 ],
[0 ,-9 ,31, -10, 0 ,0 ,0, 0 ,0 ],
[0 ,0 ,-10, 79, -30, 0 ,0 ,0, -9 ],
[0 ,0 ,0 ,-30, 57, -7, 0, -5 ,0 ],
[0 ,0 ,0 ,0 ,-7,47 ,-30 ,0, 0 ],
[0 ,0 ,0 ,0 ,0 ,-30, 41 ,0, 0 ],
[0 ,0 ,0 ,0 ,-5, 0, 0, 27 ,-2],
[0 ,0 ,0 ,-9 ,0 ,0 ,0, -2, 29 ]])

b = np.array([-15,27,-23,0,-20,12,-7, 7,10])

In [108]:
def JacobiIterate(A,b,epsilon):
    dimension = np.shape(A)
    for i in range(dimension[0]):
        diag = np.abs(A[i,i])
        other = np.sum(np.abs(A[i])) - diag
        if np.abs(diag) < other:
            return "error, not diagonally dominant"
        
    
    matrix_U = np.triu(A,1)
    matrix_L = np.tril(A,-1)
    
    diag = 1/np.diagonal(A)
    matrix_D_reverse = np.diag(diag)
    
    x_0 = np.zeros(dimension[1])
    
    error = 100
    k = 0 # 迭代步数
    max_iterate = 1000
    while error > epsilon:
        # 矩阵形式的迭代
        x_1 = np.matmul(matrix_D_reverse,(b - np.matmul(matrix_L + matrix_U, x_0)))
        error = np.abs(np.max(x_1 - x_0))
        x_0 = x_1
        
        if k > max_iterate:
            break
        k = k+1

    return x_1,k

def GaussSeidelIterate(A,b,epsilon):
    dimension = np.shape(A)
    for i in range(dimension[0]):
        diag = np.abs(A[i,i])
        other = np.sum(np.abs(A[i])) - diag
        if np.abs(diag) < other:
            return "error, not diagonally dominant"
        
    
    matrix_U = np.triu(A,1)
    matrix_L = np.tril(A,-1)
    
    diag = np.diagonal(A)
    matrix_D = np.diag(diag)
    
    x_0 = np.zeros(dimension[1])
    
    k = 0 # 迭代步数
    error = 100
    max_iterate = 1000
    while error > epsilon:
        # 回代步骤以线性方程组的求解完成
        x_1 = linalg.solve(matrix_L + matrix_D, np.matmul(-matrix_U,x_0) + b)
        error = np.abs(np.max(x_1 - x_0))
        x_0 = x_1
        
        if k > max_iterate:
            break
        k = k+1

    return x_1, k

def SORIterate(A,b,omega,epsilon):
    dimension = np.shape(A)
    for i in range(dimension[0]):
        diag = np.abs(A[i,i])
        other = np.sum(np.abs(A[i])) - diag
        if np.abs(diag) < other:
            return "error, not diagonally dominant"
        
    
    matrix_U = np.triu(A,1)
    matrix_L = np.tril(A,-1)
    
    diag = np.diagonal(A)
    matrix_D = np.diag(diag)
    
    x_0 = np.zeros(dimension[1])
    
    k = 0 # 迭代步数
    error = 100
    max_iterate = 1000
    while error > epsilon:
        # 回代步骤以线性方程组的求解完成
        x_1 = linalg.solve(omega * matrix_L + matrix_D, omega * b - omega * np.matmul(matrix_U,x_0) + (1 - omega) * np.matmul(matrix_D,x_0))
        error = np.abs(np.max(x_1 - x_0))
        x_0 = x_1
        
        if k > max_iterate:
            break
        k = k+1

    return x_1, k

In [109]:
JacobiIterate(np.array([[1,2],[2,3]]),np.array([1,2]),0.000001)

print(JacobiIterate(matrix_A,b,0.0000001))
print(GaussSeidelIterate(matrix_A,b,0.0000001))
print(SORIterate(matrix_A,b,1.2,0.0000001))

(array([-0.28923376,  0.3454357 , -0.7128117 , -0.22060852, -0.43040041,
        0.15430872, -0.05782281,  0.20105389,  0.29022867]), 46)
(array([-0.28923291,  0.34543642, -0.71281135, -0.22060823, -0.43040015,
        0.15430926, -0.0578225 ,  0.20105396,  0.29022875]), 18)
(array([-0.28923391,  0.34543569, -0.71281175, -0.22060851, -0.43040043,
        0.15430874, -0.05782287,  0.20105389,  0.29022866]), 12)


认为最佳松弛因子是使得迭代步数最小的因子

In [110]:
omega_list = np.arange(0.02,2,0.02)

def SOReasy(omega):
    return SORIterate(matrix_A,b,omega,0.0000001)


result = [SOReasy(omega) for omega in omega_list]

In [111]:
solution = [result[i][0] for i in range(len(result))]
steps = [result[i][1] for i in range(len(result))]

for i in range(len(result)):
    print("omega = ", omega_list[i], "solution = ", solution[i], "step = ", steps[i])

omega =  0.02 solution =  [-0.26542393  0.38091272 -0.6837481  -0.19668785 -0.40639073  0.16313355
 -0.05136837  0.20886129  0.30209036] step =  282
omega =  0.04 solution =  [-0.26515727  0.38129984 -0.68341851 -0.19646601 -0.40626298  0.16315733
 -0.05134644  0.20885114  0.30212138] step =  139
omega =  0.06 solution =  [-0.28919626  0.34547105 -0.7127905  -0.22059388 -0.43038263  0.15434493
 -0.05778647  0.20105927  0.29023561] step =  444
omega =  0.08 solution =  [-0.28920599  0.34546173 -0.71279617 -0.22059779 -0.43038736  0.15433563
 -0.05779588  0.20105782  0.29023373] step =  342
omega =  0.1 solution =  [-0.28921151  0.34545647 -0.71279937 -0.22059998 -0.43039003  0.1543303
 -0.05780129  0.20105701  0.29023267] step =  278
omega =  0.12000000000000001 solution =  [-0.28921513  0.34545303 -0.71280145 -0.22060141 -0.43039177  0.15432677
 -0.05780487  0.20105648  0.29023198] step =  234
omega =  0.13999999999999999 solution =  [-0.28921784  0.34545047 -0.71280299 -0.22060248 -0.

In [119]:
print(steps[len(steps)-5])
print(result[len(steps)-5])

1001
(array([-0.28911554,  0.34555274, -0.7127807 , -0.22062554, -0.43045448,
        0.15433016, -0.05783072,  0.20103502,  0.29019265]), 1001)


In [78]:
print("最小迭代步数为 ",min(steps)," ，对应松弛因子 ",(steps.index(min(steps)) + 1)/50)

最小迭代步数为  11  ，对应松弛因子  1.16
